In [1]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

In [2]:
# Parameters
IMG_SIZE: tuple[int, int] = (512, 512)
BATCH_SIZE: int = 8
EPOCHS: int = 30
BASE_DIR: str = "./brain_scans" # Root folder containing glioma, meningioma, and pituitary_tumor folders

In [3]:
# Data Augmentation and Image Loading
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
    validation_split=0.2 # 20% data for validation
)

In [4]:
train_gen = datagen.flow_from_directory(
    BASE_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training" # Set for training data
)

Found 2452 images belonging to 3 classes.


In [5]:
val_gen = datagen.flow_from_directory(
    BASE_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation" # Set for validation data
)

Found 612 images belonging to 3 classes.


In [6]:
# Load Pre-trained ResNet50 Model (without the top layers)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(*IMG_SIZE, 3))
# base_model = ResNet50(weights='imagenet', include_top=False)

In [ ]:
# Freezing the convolutional base
for layer in base_model.layers:
    layer.trainable = False

In [7]:
# Adding new custom layers
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation="relu")(x)
x = Dropout(0.5)(x)
output = Dense(3, activation="softmax")(x) # 3 classes for glioma, meningioma, pituitary tumor

In [8]:
model = Model(inputs=base_model.input, outputs=output)

In [9]:
# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

In [10]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

In [11]:
# Train the model
history = model.fit(
    train_gen,
    epochs=EPOCHS,
    validation_data=val_gen,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

Epoch 1/30
307/307 [==============================] - 3183s 10s/step - loss: 7.4566 - accuracy: 0.4719 - val_loss: 1.0950 - val_accuracy: 0.4657 - lr: 0.0010
Epoch 2/30
307/307 [==============================] - 3150s 10s/step - loss: 0.8787 - accuracy: 0.6252 - val_loss: 4.6196 - val_accuracy: 0.3039 - lr: 0.0010
Epoch 3/30
307/307 [==============================] - 3164s 10s/step - loss: 0.7320 - accuracy: 0.6896 - val_loss: 1.3554 - val_accuracy: 0.4837 - lr: 0.0010
Epoch 4/30
307/307 [==============================] - 3147s 10s/step - loss: 0.6494 - accuracy: 0.7227 - val_loss: 1.0271 - val_accuracy: 0.5196 - lr: 0.0010
Epoch 5/30
307/307 [==============================] - 3137s 10s/step - loss: 0.7237 - accuracy: 0.7076 - val_loss: 1.6666 - val_accuracy: 0.5131 - lr: 0.0010
Epoch 6/30
307/307 [==============================] - 3122s 10s/step - loss: 0.6215 - accuracy: 0.7488 - val_loss: 0.8382 - val_accuracy: 0.6830 - lr: 0.0010
Epoch 7/30
307/307 [==============================] 

In [12]:
model.save("brain_tumor_classifier.keras")

In [13]:
val_loss, val_accuracy = model.evaluate(val_gen)

77/77 [==============================] - 175s 2s/step - loss: 0.2882 - accuracy: 0.8840


In [14]:
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

Validation Accuracy: 88.40%
